In [1]:
team = ['Run', 'Build']
job_Level = ['Intern', 'Junior', 'Intermediate']
job_Role = ['DevOps Engineer', 'App Developer', 'UX Designer', 'Data Engineer', 'ML Engineer', 'Data Scientist', 'QA Developer', 'Product Manager', 'Business Analyst']

In [2]:
from itertools import product
data = []
for comb in list(product(team, job_Level, job_Role)):
  data.append(comb)

In [13]:
len(data)

54

In [3]:
import random
random.seed(42)
adjectives = ['Awesome', 'Innovative', 'Fantastic', 'Sleek', 'Dynamic', 'Creative', 'Power', 'Wonder', 'Cool', 'Smart', 'Fun']
nouns = ['App', 'Solution', 'Hub', 'Tool', 'Platform', 'Phase', 'Matrix', 'Space']

app_names = list(product(adjectives, nouns))
app_names = [' '.join(tup) for tup in app_names]

In [12]:
len(app_names)

88

In [14]:
app_names_subset = random.sample(app_names, 50)

In [19]:
import pandas as pd
df = pd.DataFrame(data, columns = ['team', 'job_Level', 'job_Role'])

# group1 = ['DevOps Engineer', 'App Developer', 'UX Designer', 'ML Engineer', 'Data Engineer']
# group2 = ['Data Scientist', 'QA Developer', 'Product Manager', 'Business Analyst']
# df1 = df[(df['team'] == 'Run') & (df['job_Role'].isin(group1))]
# df2 = df[(df['team'] == 'Build') & (df['job_Role'].isin(group2))]

# final = pd.concat([df1, df2]).reset_index(drop=True)
dfs = pd.concat([df, df, df, df, df, df]).reset_index(drop=True)

rows = len(dfs)
dfs['app_name'] = random.choices(app_names_subset, k = rows)

In [20]:
new_df = dfs.groupby(['team', 'job_Level', 'job_Role'])['app_name'].apply(lambda x: ', '.join(set(x))).reset_index()

In [21]:
new_df.head(2)

,team,job_Level,job_Role,app_name
0,Build,Intermediate,App Developer,"Fantastic Matrix, Innovative Matrix, Fantastic..."
1,Build,Intermediate,Business Analyst,"Awesome Phase, Fun Hub, Smart Matrix, Power To..."


In [22]:
new_df.to_excel('query.xlsx')

In [23]:
len(dfs)

324

In [24]:
dfs_unique= dfs.drop_duplicates().reset_index(drop=True)
dfs_unique.to_excel('dfs_unique.xlsx')
dfs_unique['Decision'] = 1

In [25]:
dfs_unique

,team,job_Level,job_Role,app_name,Decision
0,Run,Intern,DevOps Engineer,Fantastic Tool,1
1,Run,Intern,App Developer,Power Solution,1
2,Run,Intern,UX Designer,Dynamic App,1
3,Run,Intern,Data Engineer,Power Space,1
4,Run,Intern,ML Engineer,Smart Space,1
...,...,...,...,...,...
302,Build,Intermediate,Data Engineer,Fun Tool,1
303,Build,Intermediate,ML Engineer,Awesome Platform,1
304,Build,Intermediate,Data Scientist,Sleek Hub,1
305,Build,Intermediate,Product Manager,Awesome App,1


### Data Transformation and Create labels to generate data set


In [26]:
dfs_neg = dfs_unique.copy()
dfs_neg = dfs_neg.groupby(['team', 'job_Level', 'job_Role'])['app_name'].apply(lambda x: list(x)).reset_index()

def list_reduction(a, b):
  return [item for item in a if item not in b ]

dfs_neg['app_name'] = dfs_neg['app_name'].apply(lambda x: list_reduction(app_names_subset, x))
dfs_neg['app_name'] = dfs_neg['app_name'].apply(lambda x: random.sample(x, 5))
dfs_neg = dfs_neg.reset_index(drop=True)

import numpy as np
lol = dfs_neg[['team', 'job_Level', 'job_Role']].values.tolist()
lol = [row for row in lol for _ in range(5)]
app_name_list = [i for a in dfs_neg['app_name'] for i in a]
concat_list = [sub + [add] for sub, add in zip(lol, app_name_list)]
df_dd = pd.DataFrame(concat_list, columns = ['team', 'job_Level', 'job_Role', 'app_name']).drop_duplicates().reset_index(drop=True)
df_dd['Decision'] = 0

In [27]:
df_dd

,team,job_Level,job_Role,app_name,Decision
0,Build,Intermediate,App Developer,Awesome Tool,0
1,Build,Intermediate,App Developer,Creative Solution,0
2,Build,Intermediate,App Developer,Creative Platform,0
3,Build,Intermediate,App Developer,Dynamic Solution,0
4,Build,Intermediate,App Developer,Dynamic Tool,0
...,...,...,...,...,...
265,Run,Junior,UX Designer,Creative Phase,0
266,Run,Junior,UX Designer,Cool App,0
267,Run,Junior,UX Designer,Fantastic Matrix,0
268,Run,Junior,UX Designer,Creative App,0


In [28]:
complete_table = pd.concat([dfs_unique, df_dd]).reset_index(drop=True)

In [29]:
complete_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577 entries, 0 to 576
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   team       577 non-null    object
 1   job_Level  577 non-null    object
 2   job_Role   577 non-null    object
 3   app_name   577 non-null    object
 4   Decision   577 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 22.7+ KB


In [30]:
complete_table

,team,job_Level,job_Role,app_name,Decision
0,Run,Intern,DevOps Engineer,Fantastic Tool,1
1,Run,Intern,App Developer,Power Solution,1
2,Run,Intern,UX Designer,Dynamic App,1
3,Run,Intern,Data Engineer,Power Space,1
4,Run,Intern,ML Engineer,Smart Space,1
...,...,...,...,...,...
572,Run,Junior,UX Designer,Creative Phase,0
573,Run,Junior,UX Designer,Cool App,0
574,Run,Junior,UX Designer,Fantastic Matrix,0
575,Run,Junior,UX Designer,Creative App,0


In [31]:
# !pip install catboost

In [32]:
# split the training and test case
from sklearn.model_selection import train_test_split
features = ['team', 'job_Level', 'job_Role', 'app_name']
X = complete_table[features]
y = complete_table['Decision']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.33, random_state=42)

In [33]:
X.values

array([['Run', 'Intern', 'DevOps Engineer', 'Fantastic Tool'],
       ['Run', 'Intern', 'App Developer', 'Power Solution'],
       ['Run', 'Intern', 'UX Designer', 'Dynamic App'],
       ...,
       ['Run', 'Junior', 'UX Designer', 'Fantastic Matrix'],
       ['Run', 'Junior', 'UX Designer', 'Creative App'],
       ['Run', 'Junior', 'UX Designer', 'Awesome Tool']], dtype=object)

### Catboost model to decide if an app is needed for a certain job role at a specific job level

In [41]:
from catboost import CatBoostClassifier
# from sklearn.metrics import classification_report
cat_features = list(range(X.shape[1]))
clf = CatBoostClassifier(
    depth = 1,
    iterations=20000,
    learning_rate = 0.0001
)
clf.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
)

# param_grid = {
#     'learning_rate': [0.1, 0.01, 0.001],
#     'iterations': [60, 70]
# }

# print("Best Hyperparameters -> ", grid_search.best_params_)
# print("Best Score -> ", grid_search.best_score_)

0:	learn: 0.6931463	test: 0.6931461	best: 0.6931461 (0)	total: 3.38ms	remaining: 1m 7s
1:	learn: 0.6931457	test: 0.6931455	best: 0.6931455 (1)	total: 4.53ms	remaining: 45.3s
2:	learn: 0.6931453	test: 0.6931450	best: 0.6931450 (2)	total: 5.31ms	remaining: 35.4s
3:	learn: 0.6931450	test: 0.6931445	best: 0.6931445 (3)	total: 6.17ms	remaining: 30.8s
4:	learn: 0.6931442	test: 0.6931440	best: 0.6931440 (4)	total: 7.07ms	remaining: 28.3s
5:	learn: 0.6931440	test: 0.6931436	best: 0.6931436 (5)	total: 7.77ms	remaining: 25.9s
6:	learn: 0.6931434	test: 0.6931429	best: 0.6931429 (6)	total: 8.37ms	remaining: 23.9s
7:	learn: 0.6931430	test: 0.6931421	best: 0.6931421 (7)	total: 10.6ms	remaining: 26.4s
8:	learn: 0.6931427	test: 0.6931416	best: 0.6931416 (8)	total: 11.2ms	remaining: 24.9s
9:	learn: 0.6931424	test: 0.6931412	best: 0.6931412 (9)	total: 13ms	remaining: 25.9s
10:	learn: 0.6931420	test: 0.6931406	best: 0.6931406 (10)	total: 14ms	remaining: 25.5s
11:	learn: 0.6931418	test: 0.6931402	best: 0.

Here are some eval_metrics, but we can do label encoding and GridCV to improve the model

In [43]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

pred_proba = clf.predict_proba(X_test)
pred_labels = np.argmax(pred_proba, axis=1)

accuracy = accuracy_score(y_test, pred_labels)
precision = precision_score(y_test, pred_labels)
recall = recall_score(y_test, pred_labels)
f1 = f1_score(y_test, pred_labels)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5287958115183246
Precision: 0.5406976744186046
Recall: 0.8942307692307693
F1 Score: 0.6739130434782609


In [38]:
pred_proba

array([[0.47634918, 0.52365082],
       [0.49413471, 0.50586529],
       [0.48509568, 0.51490432],
       [0.48195641, 0.51804359],
       [0.4726182 , 0.5273818 ],
       [0.49289178, 0.50710822],
       [0.47139487, 0.52860513],
       [0.48679802, 0.51320198],
       [0.48702431, 0.51297569],
       [0.46555628, 0.53444372],
       [0.47358045, 0.52641955],
       [0.47437201, 0.52562799],
       [0.46201237, 0.53798763],
       [0.49850762, 0.50149238],
       [0.50296291, 0.49703709],
       [0.48174443, 0.51825557],
       [0.48237377, 0.51762623],
       [0.47430556, 0.52569444],
       [0.48234244, 0.51765756],
       [0.47815606, 0.52184394],
       [0.49979164, 0.50020836],
       [0.4878076 , 0.5121924 ],
       [0.48189182, 0.51810818],
       [0.50620223, 0.49379777],
       [0.4803347 , 0.5196653 ],
       [0.46677256, 0.53322744],
       [0.47247293, 0.52752707],
       [0.4879204 , 0.5120796 ],
       [0.4876512 , 0.5123488 ],
       [0.4698103 , 0.5301897 ],
       [0.

In [39]:
X_test

,team,job_Level,job_Role,app_name
118,Run,Junior,App Developer,Creative Solution
393,Build,Intern,UX Designer,Creative Matrix
514,Run,Intern,ML Engineer,Dynamic App
101,Build,Intermediate,UX Designer,Wonder Phase
437,Build,Junior,UX Designer,Smart Platform
...,...,...,...,...
423,Build,Junior,ML Engineer,Creative App
114,Run,Intern,QA Developer,Sleek Hub
141,Build,Intern,Business Analyst,Awesome App
552,Run,Junior,DevOps Engineer,Creative Tool


Recall Rate matters since we want to give service approved to the right group of employees.

In [82]:
import pickle
with open("catboost.pkl", 'wb') as f:
   pickle.dump(clf, f)

In [83]:
X1 = [['Build', 'Intern', 'Business Analyst', 'Smart Tool']]

In [84]:
pd.DataFrame(X1)

,0,1,2,3
0,Build,Intern,Business Analyst,Smart Tool\t


In [85]:
clf.predict(pd.DataFrame(X1, columns=['team', 'job_Level', 'job_Role', 'app_name']))[0] == 1

True